In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install SimpSOM

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sb
import SimpSOM as sps
from imageio import imwrite
import matplotlib.pyplot as plot
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from matplotlib.cbook import get_sample_data
%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

from sklearn.preprocessing import StandardScaler
from PIL import Image, ImageChops
data=pd.read_csv('../input/iris-flower-dataset/IRIS.csv')
#first we check the data 
data.head();
data.describe();

In [ ]:
#we plot the data into some bar graphs that show the correlation between the different attributes of the irises 
#1-->a barplot showing the ris data
plot.figure(figsize=(12,6))
plot.title('Barplot for Iris Data', fontsize=10)
sb.barplot(data=data)
#2--> a barplot showing correltaions between the different attributes of the iris data 
label = ['SEPAL LENGTH', 'SEPAL WIDTH', 'PETAL LENGTH', 'PETAL WIDTH']
column = data.columns[:-1].tolist()
plot.figure(figsize=(12,6))
plot.title('Barplot for {}'.format(label[0]), fontsize=10)
sb.barplot(x=column[0], y='species', data=data, color='blue')
plot.xlabel(label[0], fontsize=12)
plot.ylabel('SPECIES', fontsize=12)
#3-->
plot.figure(figsize=(12,6))
plot.title('Barplot for {}'.format(label[1]), fontsize=10)
sb.barplot(x=column[1], y='species', data=data, color='yellow')
plot.xlabel(label[1], fontsize=12)
plot.ylabel('SPECIES', fontsize=12)
#4-->
plot.figure(figsize=(12,6))
plot.title('Barplot for {}'.format(label[2]), fontsize=10)
sb.barplot(y=column[2], x='species', data=data, color='red')
plot.xlabel(label[2], fontsize=12)
plot.ylabel('SPECIES', fontsize=12)
#5-->
plot.figure(figsize=(12,6))
plot.title('Barplot for {}'.format(label[3]), fontsize=10)
sb.barplot(y=column[3], x='species', data=data, color='lightblue')
plot.xlabel(label[3], fontsize=12)
plot.ylabel('SPECIES', fontsize=12)
plot.show()

In [ ]:
#printing the count plot for the species of the flowers 
#according to the data 
plot.figure(figsize=(10,5))
plot.title('Countplot for the Data', fontsize=15)
sb.countplot(x=data.species);



In [ ]:
# now we need to do the histogram for the data, which shows a correlation between the attributes values and the count of each value 
plot.figure(figsize=(12,6))
plot.title('Histplot for the IRIS Data', fontsize=15)
sb.histplot(data)
plot.show

In [ ]:
#we visualize the data as a pie chart in this section, the amount list has the count of each iris 
labels = ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']
sizes = [50,50,50]
colors = ['lightskyblue', 'lightcoral', 'orange'] 
explode = (0, 0, 0)

plot.pie(sizes,              # data
       explode=explode,    # offset parameters 
        labels=labels,      # slice labels
        colors=colors,      # array of colours
        autopct='%1.1f%%',  # print the values inside the wedges
        shadow=True,        # enable shadow
        startangle=70       # starting angle
        )
plot.show()

In [ ]:
#in order to see the differences between each iris, we use a scatter plot to cluster the similar ones according to their attributes
plot.figure(figsize=(10,5))
plot.title('Scatterplot for the Iris data', fontsize=15)
sb.scatterplot(data=data, s=100)
plot.show()

In [ ]:
#we use a correlation matrix in the form of a heatmap to show the relations between each column 
plot.figure(figsize=(8,6))
plot.title('Heatmap of the Correlations between Iris Data', fontsize=10)
sb.heatmap(data=data[column].corr(), annot=True, cmap='YlGnBu')
plot.show()

In [ ]:
#after visualizing the data, we want to cluster the irises using SOM, this notebook uses the Unsupervised Learning with SOM kaggle notebook 

#first we start by seeding

np.random.seed(0)
# we get the data and we prepare a training set from it, we take 70 from the 150
iris=data
iris = iris.sample(n=70, random_state=0,replace='true')
iris.info()
#then we need to normalize the data, which is one of the most important stages of an SOM 
#we need to use numerical data for the categories that we have 
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder=OrdinalEncoder()
iris_encoded=ordinal_encoder.fit_transform(iris[['species']])
# after we use numerical data for the categorization, we normalize the data 
iris_norm = StandardScaler().fit_transform(iris_encoded)


In [ ]:
#we build a network to cluster the data
net = sps.somNet(20, 20, iris_norm, PBC=True, PCI=True)
net.train(0.1, 10000)
net.diff_graph(show=True,printout=True)



In [ ]:
# some functions are defined here to make it easier to cluster and visualize the data 
#taken from the SOM project mentioned above
def autocrop(fileName):
    im = Image.open(fileName)
    im=im.crop((0,100,2900,im.size[1]))
    bg = Image.new(im.mode, im.size, im.getpixel((0,0)))
    diff = ImageChops.difference(im, bg)
    diff = ImageChops.add(diff, diff, 2.0, -100)
    bbox = diff.getbbox()
    if bbox:
        return im.crop(bbox)

def posMap(x,y):
     if y%2==0:
        return [y, x*2/np.sqrt(3)*3/4]
     else:
        return [y+0.5, x*2/np.sqrt(3)*3/4]
    
def posCount(x,y):
     return y*40+x

def posCountR(x):
     return [np.int(x%40),np.int(x/40)]

In [ ]:
#we print a couple of example nodes here to see the progress 
listNodes=[[20,0],[23,11],[1,6],[13,37],[7,33],[18,31]]
listCount=[posCount(20,0), posCount(23,11), posCount(1,6), posCount(13,37), posCount(7,33), posCount(18,31)]

i=0
for node in net.nodeList:
    if i in listCount:
        print('Node\'s position: {:d} {:d}'.format(posCountR(i)[1], posCountR(i)[0]) )
        plot.imshow(np.asarray(node.weights).reshape(28,28))
        plot.axis('off')
        plot.show()
    i+=1
    
projData=net.project(trainSt[:500])

In [ ]:
#we take a cropped version of the map 
cropped = autocrop('nodesDifference.png')
cropped.save('cropped.png')
#then we use a scatter plot to visualize the data 
#And here we prepare the plotly graph. 
trace0 = go.Scatter(
    x = [x for x,y in projData],
    y = [y for x,y in projData],
#    name = labels,
    hoveron = [str(n) for n in labels],
    text = [str(n) for n in labels],
    mode = 'markers',
    marker = dict(
        size = 8,
        color = labels,
        colorscale ='Jet',
        showscale = False,
        opacity = 1
    ),
    showlegend = False

)
data = [trace0]

layout = go.Layout(
    images= [dict(
                  source= "cropped.png",
                  xref= "x",
                  yref= "y",
                  x= -0.5,
                  y= 39.5*2/np.sqrt(3)*3/4,
                  sizex= 40.5,
                  sizey= 40*2/np.sqrt(3)*3/4,
                  sizing= "stretch",
                  opacity= 0.5,
                  layer= "below")],
    width = 800,
    height = 800,
    hovermode= 'closest',
    xaxis= dict(
        range=[-1,41],
        zeroline=False,
        showgrid=False,
        ticks='',
        showticklabels=False
    ),
    yaxis=dict(
        range=[-1,41],
        zeroline=False,
        showgrid=False,
        ticks='',
        showticklabels=False
    ),
    showlegend= True
)


fig = dict(data=data, layout=layout)
py.iplot(fig, filename='styled-scatter')